![title](imagens/01.jpg)

# Modelo de Regressão linear - Airbnb Nova York



**Contexto**


Desde 2008, os hóspedes e anfitriões têm usado o Airbnb para expandir as possibilidades de viagem e apresentar uma forma mais única e personalizada de experimentar o mundo. Este conjunto de dados descreve a atividade de listagem e as métricas em NYC, NY para 2019.



**Conteúdo**

Este arquivo de dados inclui todas as informações necessárias para descobrir mais sobre hosts, disponibilidade geográfica, métricas necessárias para fazer previsões e tirar conclusões.




**Reconhecimentos**

Este conjunto de dados públicos faz parte do Airbnb, e a fonte original pode ser encontrada neste site .

**Inspiração**

O que podemos aprender sobre diferentes hosts e áreas?

O que podemos aprender com as previsões? (ex: locais, preços, comentários, etc)

Quais hosts são os mais ocupados e por quê?

Existe alguma diferença perceptível de tráfego entre as diferentes áreas e qual poderia ser o motivo?

In [ ]:
!pip install -q -U watermark

In [ ]:
from platform import python_version

print('Versão python neste Jupyter Notebook:', python_version())

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
%reload_ext watermark
%watermark -a "Rafael Gallo" --iversions

In [ ]:
sns.set_palette("Accent")
sns.set_style('darkgrid')

In [ ]:
df = pd.read_csv("AB_NYC_2019.csv")
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df.duplicated().sum()

In [ ]:
df.host_id.value_counts().iloc[:5]

In [ ]:
df.columns = ["ID",
             "Nome",
             "hospede_id",
             "hospede_nome",
             "Bairro",
             "Vizinhança",
             "latitude",
             "longitude",
             "Tipo de sala",
             "Preço",
             "Noites mínimas",
             "Número de comentários",
             "Última revisão",
             "Avaliações por mês",
             "Contagem de listagens de host calculada",
             "Disponibilidade 365"]

df.head()

In [ ]:
df.info()

In [ ]:
# Média

print("Média preços", df.Preço.mean())

In [ ]:
# Mediana

print("Mediana preços", df.Preço.median())

In [ ]:
# Moda

print("Moda preços", df.Preço.mode())

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize = (10,8))

corr = df.corr()
sns.heatmap(corr,  cmap = 'plasma', annot= True);

# Análise exploratória de dados

In [ ]:
plt.figure(figsize=(18, 8))

sns.countplot(df["Bairro"])

In [ ]:
plt.figure(figsize = (18, 8))

ax = sns.barplot(x='Preço', y='Bairro', data= df,
                palette = 'plasma')

ax.set_title('Média Preço por Distrito')

In [ ]:
plt.figure(figsize=(18, 8))

sns.countplot(data = df, x = "Tipo de sala", hue = "Bairro")

In [ ]:
plt.figure(figsize=(18, 8))

sns.boxplot(x = "Bairro", y = "Disponibilidade 365", data = df)

In [ ]:
plt.figure(figsize=(18, 8))

ax = sns.scatterplot(x="Preço", y="Noites mínimas", hue = 'Bairro', data=df)
ax.set_title('Condomínio x Preco')
ax.set_ylabel('Condomínio')
ax.set_xlabel('Preco')

In [ ]:
plt.figure(figsize=(10, 8))

sns.lineplot(x = "Noites mínimas", y = "Preço", data = df)

In [ ]:
plt.figure(figsize=(10, 8))

sns.lineplot(x = "Avaliações por mês", y = "Preço", data = df)

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(x = "latitude", y = "longitude", data = df, hue = "Bairro")

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(x = "latitude", y = "longitude", data = df, hue = "Tipo de sala")

In [ ]:
apt = df[["latitude", "longitude", "Preço"]]

fig = px.scatter_mapbox(apt,
                       lat = "latitude",
                       lon = "longitude",
                       size = "Preço",
                       color_continuous_scale = px.colors.cyclical.IceFire, 
                       size_max = 15, 
                       zoom = 10)

fig.update_layout(
        title = 'Mapa região de apartamentos Nova York',
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
apt = df[["latitude", "longitude", "Disponibilidade 365"]]

fig = px.scatter_mapbox(apt,
                       lat = "latitude",
                       lon = "longitude",
                       size = "Disponibilidade 365",
                       color_continuous_scale = px.colors.cyclical.IceFire, 
                       size_max = 15, 
                       zoom = 10)

fig.update_layout(
        title = 'Mapa região de apartamentos Nova York',
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
sns.pairplot(df)

# Limpeza de dados

In [ ]:
df.info()

In [ ]:
df.drop(["ID", "Nome", "latitude", "longitude", "hospede_id", "Última revisão"], axis = 1, inplace = True)
df.head()

In [ ]:
df.drop(["hospede_nome"], axis = 1, inplace = True)
df

In [ ]:
df.isna().sum()

In [ ]:
df["Avaliações por mês"] = df["Avaliações por mês"].fillna(0)
df

In [ ]:
df.isna().sum()

In [ ]:
x = df[["Preço"]].values.reshape(-1,1)
y = df["Preço"].values.reshape(-1,1)

In [ ]:
x

In [ ]:
y

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler_x = scaler.fit_transform(x)
scaler_y = scaler.fit_transform(y)

In [ ]:
scaler_x

In [ ]:
scaler_y

In [ ]:
scaler_x.shape

In [ ]:
scaler_y.shape

# Treino e Teste

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaler_x, scaler_y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

# Modelo de regressão linear

In [ ]:
from sklearn.linear_model import LinearRegression

reg_line = LinearRegression()
reg_line.fit(X_train, y_train)
reg_pred = reg_line.predict(X_train)
reg_pred

In [ ]:
reg_line.score(X_train, y_train)

In [ ]:
reg_line.coef_

In [ ]:
reg_line.coef_ * 27.74456356 + reg_line.intercept_

In [ ]:
pred = reg_line.predict(X_train)
pred

In [ ]:
y_pred = reg_line.predict(X_test)
y_pred

In [ ]:
pred2 = y_train - pred
pred2

In [ ]:
plt.figure(figsize=(18, 8))
plt.scatter(pred, y_train)
plt.plot(pred, reg_line.predict(X_train), color = "red")
plt.title("Grafico de regressão linear - AirBnb NY", fontsize = 20)
plt.xlabel("Valor")
plt.ylabel("Valor do imóvel")
plt.legend(["Valor", "Imóvel"])

In [ ]:
ax = sns.distplot(pred)
ax.figure.set_size_inches(20, 8)
ax.set_title('Distribuição de Frequências dos Resíduos', fontsize=18)
ax.set_xlabel('Imóvel', fontsize=14)
ax

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn import metrics
from math import sqrt

print('MSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2", r2_score(y_test, y_pred))

# Previsão do imovel

In [ ]:
X_test[0:1]

In [ ]:
a1 = X_test[0: 1]
a1

In [ ]:
pred = reg_line.predict(a1)[0]
pred

# Salvando o modelo de regressão linear

In [ ]:
import pickle

saida = open("modelo_previsao_imóvel_AirBnb", "wb")
saida.close()